<a href="https://colab.research.google.com/github/Tahimi/convectionDiffusion/blob/main/lagoGeoFileMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
meshInputFileName = '/content/drive/MyDrive/UFAL/advection_diffusion/FiniteDiference_FipyFiniteVolume/lagoBoundaryVerticesSIRGAS2000UTMzone24S.csv'

import pandas as pd

df = pd.read_csv(meshInputFileName, usecols = ['x', 'y'])

xmin, ymin = min(df['x']), min(df['y'])
df['x'], df['y'] = df['x']-xmin, df['y']-ymin
x, y = df['x'], df['y']
print('xmin = ', xmin, ', ymin = ', ymin, ', len(x) = len(y) = ', len(x))
df.head(4)

xmin =  753160 , ymin =  8920062 , len(x) = len(y) =  285


,x,y
0,893,0
1,898,5
2,924,32
3,928,36


In [3]:
import numpy as np

def getSelectedVerticesIdxs(dist_min):
    global x, y
    l = []
    i = 0
    i_max = len(x)
    # select the first vertice
    l.append(i)
    while i < i_max:
        a = np.array([x[i], y[i]])
        j = 1
        # advance the increment j untill enconter the next distante enough vertice
        while i+j < len(x) and np.linalg.norm(a - np.array([x[i+j], y[i+j]])) <= dist_min:
            j += 1
        if i+j >= i_max:
            break
        # the encountred vertice has index = i + the increment j
        i += j
        # adding this vertice to the selection
        l.append(i)
    # check the distant between the last and first vertices
    if np.linalg.norm(a - np.array([x[0], y[0]])) <= dist_min: l.pop(-1)
    return l

dist_min = 30.
selectedVerticesIdxs = getSelectedVerticesIdxs(dist_min)
print('len(selectedVerticesIdxs) = ', len(selectedVerticesIdxs))

len(selectedVerticesIdxs) =  183


In [4]:
geoFileName = '/content/drive/MyDrive/UFAL/advection_diffusion/FiniteDiference_FipyFiniteVolume/lagoGeoFileForGmsh.geo.txt'

def writeGeoFile(fileName = geoFileName):
    global selectedVerticesIdxs, x, y
    with open(fileName, 'w') as geoFile:
        geoFile.write('lc = '+str(dist_min)+'; // characteristic length of mesh elements\n')
        geoFile.write('\n// defining the points\n')
        pointIdx = 0
        for i in selectedVerticesIdxs:
            pointIdx += 1
            txtLine = 'Point('+str(pointIdx)+') = {'+str(x[i])+','+str(y[i])+','+str(0.)+',lc};\n'
            geoFile.write(txtLine)
            pointsNbr = pointIdx

        geoFile.write('\n// defining the lines\n')
        linesNbr = pointsNbr
        for lineIdx in range(1, linesNbr+1):
            pL = lineIdx; pR = lineIdx+1
            if lineIdx == linesNbr: pR = 1
            txtLine = 'Line('+str(lineIdx)+') = {'+str(pL)+','+str(pR)+'};\n'
            geoFile.write(txtLine)

        geoFile.write('\n// defining the line loop and the plane surface\n')
        geoFile.write('Line Loop(1) = {')
        [geoFile.write(str(i)+',') for i in range(1,linesNbr)]
        geoFile.write(str(linesNbr)+'};\n')
        geoFile.write('Plane Surface(1) = {1};\n')

        geoFileFinalParte='''
// Define the meshing algorithm for the surface
Mesh.Algorithm = 8; // Frontal-Delaunay for quads

// Define the number of cells for the mesh
Mesh.RecombinationAlgorithm = 2; // full-quad recombination algorithm
Mesh.RecombineAll = 1;
Mesh.Smoothing = 10;
Color Grey50{ Surface{ 1 }; }

// Define the physical groups
Physical Surface("Lagoa") = {1};'''
        geoFile.write(geoFileFinalParte)
        print('successfully generated file:', geoFileName)

writeGeoFile()

successfully generated file: /content/drive/MyDrive/UFAL/advection_diffusion/FiniteDiference_FipyFiniteVolume/lagoGeoFileForGmsh.geo.txt
